In [ ]:
"""
Analytic Hierarchy Process (AHP) Analysis Tool
==============================================

This tool performs AHP analysis on pairwise comparison matrices to calculate
criteria weights and assess consistency. It supports multiple expert judgments
and calculates geometric mean weights.

Requirements:
- numpy
- scipy
- pandas
"""

import numpy as np
from scipy import linalg
import pandas as pd


class AHPAnalyzer:
    """
    A class for performing Analytic Hierarchy Process (AHP) analysis.
    
    The AHP method helps in multi-criteria decision making by converting
    subjective comparisons into quantitative weights.
    """
    
    # Random Index values for consistency ratio calculation
    RANDOM_INDEX = {
        1: 0, 2: 0, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32,
        8: 1.41, 9: 1.45, 10: 1.49, 11: 1.51, 12: 1.54, 13: 1.56,
        14: 1.57, 15: 1.58
    }
    
    def __init__(self, criteria_labels):
        """
        Initialize the AHP analyzer with criteria labels.
        
        Args:
            criteria_labels (list): List of criterion names
        """
        self.criteria_labels = criteria_labels
        self.n_criteria = len(criteria_labels)
    
    def calculate_weights_and_consistency(self, matrix):
        """
        Calculate priority weights and consistency ratio for a pairwise comparison matrix.
        
        Args:
            matrix (numpy.ndarray): Square pairwise comparison matrix
            
        Returns:
            tuple: (weights, consistency_ratio, lambda_max)
        """
        # Validate matrix
        if matrix.shape[0] != matrix.shape[1]:
            raise ValueError("Matrix must be square")
        if matrix.shape[0] != self.n_criteria:
            raise ValueError(f"Matrix size must match number of criteria ({self.n_criteria})")
        
        # Calculate eigenvalues and eigenvectors
        eigenvalues, eigenvectors = linalg.eig(matrix)
        
        # Find principal eigenvalue and corresponding eigenvector
        max_idx = np.argmax(np.real(eigenvalues))
        lambda_max = np.real(eigenvalues[max_idx])
        weights = np.real(eigenvectors[:, max_idx])
        
        # Normalize weights to sum to 1
        weights = np.abs(weights) / np.sum(np.abs(weights))
        
        # Calculate consistency metrics
        consistency_index = (lambda_max - self.n_criteria) / (self.n_criteria - 1)
        random_index = self.RANDOM_INDEX.get(self.n_criteria, 1.59)
        consistency_ratio = consistency_index / random_index if random_index > 0 else 0
        
        return weights, consistency_ratio, lambda_max
    
    def geometric_mean_weights(self, weights_list):
        """
        Calculate geometric mean of multiple weight vectors.
        
        Args:
            weights_list (list): List of weight arrays from multiple experts
            
        Returns:
            numpy.ndarray: Normalized geometric mean weights
        """
        if not weights_list:
            raise ValueError("weights_list cannot be empty")
        
        weights_array = np.array(weights_list)
        
        # Calculate geometric mean
        product = np.prod(weights_array, axis=0)
        n_experts = len(weights_list)
        geo_mean = product ** (1/n_experts)
        
        # Normalize to sum to 1
        return geo_mean / np.sum(geo_mean)
    
    def analyze_matrices(self, matrices, output_prefix="ahp_results"):
        """
        Analyze multiple AHP matrices and generate results.
        
        Args:
            matrices (list): List of pairwise comparison matrices
            output_prefix (str): Prefix for output files
            
        Returns:
            dict: Analysis results including weights and consistency metrics
        """
        if not matrices:
            raise ValueError("matrices list cannot be empty")
        
        results = {
            'individual_weights': [],
            'consistency_ratios': [],
            'lambda_max_values': [],
            'geometric_mean_weights': None,
            'consistency_issues': []
        }
        
        print(f"Analyzing {len(matrices)} matrices...")
        print("=" * 50)
        
        # Analyze each matrix
        for i, matrix in enumerate(matrices, 1):
            try:
                weights, cr, lambda_max = self.calculate_weights_and_consistency(matrix)
                
                results['individual_weights'].append(weights)
                results['consistency_ratios'].append(cr)
                results['lambda_max_values'].append(lambda_max)
                
                print(f"\nMatrix {i}:")
                print(f"  λ_max: {lambda_max:.4f}")
                print(f"  Consistency Ratio: {cr:.4f}")
                
                if cr > 0.1:
                    issue = f"Matrix {i} has high inconsistency (CR = {cr:.4f})"
                    results['consistency_issues'].append(issue)
                    print(f"  ⚠️  WARNING: {issue}")
                else:
                    print(f"  ✓ Acceptable consistency")
                    
            except Exception as e:
                print(f"  ❌ Error analyzing Matrix {i}: {e}")
                continue
        
        # Calculate geometric mean if we have valid results
        if results['individual_weights']:
            results['geometric_mean_weights'] = self.geometric_mean_weights(
                results['individual_weights']
            )
            
            # Save results
            self._save_results(results, output_prefix)
            self._print_summary(results)
        
        return results
    
    def _save_results(self, results, output_prefix):
        """Save results to CSV file."""
        if results['geometric_mean_weights'] is not None:
            df = pd.DataFrame({
                'Criterion': self.criteria_labels,
                'Weight': results['geometric_mean_weights']
            })
            
            filename = f"{output_prefix}_weights.csv"
            df.to_csv(filename, index=False, float_format="%.5f")
            print(f"\n📊 Results saved to: {filename}")
    
    def _print_summary(self, results):
        """Print analysis summary."""
        print("\n" + "=" * 50)
        print("FINAL WEIGHTS SUMMARY")
        print("=" * 50)
        
        if results['geometric_mean_weights'] is not None:
            for criterion, weight in zip(self.criteria_labels, results['geometric_mean_weights']):
                print(f"{criterion:<35}: {weight:.5f}")
        
        print(f"\nConsistency Summary:")
        print(f"  Average CR: {np.mean(results['consistency_ratios']):.4f}")
        print(f"  Matrices with CR > 0.1: {len(results['consistency_issues'])}")
        
        if results['consistency_issues']:
            print("\n⚠️  Consistency Issues:")
            for issue in results['consistency_issues']:
                print(f"  • {issue}")


def main():
    """
    Example usage of the AHP analyzer.
    Modify this section with your own data and criteria.
    """
    
    # Define your criteria
    criteria = [
        "Distance to Power Transmission Lines",
        "Slope", 
        "Distance to Rivers and Lakes",
        "Distance from Fault Lines",
        "Distance to Protected Areas",
        "Land Use",
        "Distance to Bird Migration Routes",
        "Distance to Transformers",
        "Aspect (Slope Direction)",
        "Altitude",
        "Distance to Settlements",
        "Solar Radiation",
        "Distance to Airports",
        "Distance to Highways"
    ]
    
    # Example matrices (replace with your actual data)
    # Note: These are placeholder matrices - replace with your real pairwise comparison data
    matrix1 = np.array([
        # Add your first pairwise comparison matrix here
        # This should be a 14x14 matrix for the 14 criteria above
    ])
    
    matrix2 = np.array([
        # Add your second pairwise comparison matrix here
    ])
    
    matrix3 = np.array([
        # Add your third pairwise comparison matrix here
    ])
    
    # You would populate these matrices with your actual data
    # For now, they're empty placeholders
    
    # Initialize analyzer
    analyzer = AHPAnalyzer(criteria)
    
    # Analyze matrices (uncomment and modify when you have real data)
    # matrices = [matrix1, matrix2, matrix3]
    # results = analyzer.analyze_matrices(matrices, output_prefix="solar_farm_siting")
    
    print("AHP Analyzer initialized successfully!")
    print("Please replace the placeholder matrices with your actual pairwise comparison data.")


if __name__ == "__main__":
    main()